<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/028_Tool_Design%26Structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### 🛠️ **Tool Design & Structure**

In this lecture, we learned that **tool descriptions** are essential for agents to understand and apply tools effectively. Instead of relying on vague commands, agents need clear, structured information about the tools they will use. Here are the core concepts we need to grasp:

---

### **1. Tool Definitions Are Essential for Action**

* **Agents cannot perform tasks** unless they understand the **tools** at their disposal.
* Each tool must be **described with clear metadata** that specifies:

  * **Tool name**: What does the tool do?
  * **Arguments (inputs)**: What does the tool require in order to function?
  * **Outputs**: What results or changes can we expect after the tool runs?

### **2. Tool Descriptions Enable Clear Interfacing**

* **JSON Schema** is a structured way of defining tool properties (inputs, outputs, constraints).

  * **Inputs** can be basic data types (e.g., strings, numbers) or more complex (e.g., arrays, enums).
  * **Outputs** can be validated for consistency and expected results.

### **3. Tools Need to Be Interpretable by the LLM**

* The **LLM** must understand when and how to use each tool.
* We need to provide **examples** and clear definitions so that the LLM can **recognize tool requirements** and **apply the correct logic**.

### **4. JSON Schema as a Tool “Contract”**

* By using JSON Schema, we define the **contract** between the agent and the tool:

  * What the tool is called
  * What it needs to function (parameters)
  * What it outputs (results)

  This contract allows the **LLM to execute the tool without confusion**, knowing what the inputs are, and how to apply them.

### **5. Extensibility and Flexibility of Tool Design**

* Tools can be **simple** (like listing files) or **complex** (e.g., analyzing sentiment in documents).
* As tools grow in complexity, you can introduce more advanced features:

  * **Enums** (for predefined options)
  * **Booleans** (e.g., active/inactive states)
  * **Arrays** (lists of files, multiple parameters)

  Flexibility in tool design ensures that your agent can evolve over time and handle a variety of tasks.

---

### **Objectives for This Notebook:**

1. **Understand the structure of tool descriptions** using JSON.
2. **Build several examples** of tools with varied complexity:

   * Basic tools with simple inputs/outputs
   * Tools requiring lists, boolean flags, and enums
3. **Understand how to format and define each tool clearly** to make it LLM-friendly and executable.

By the end of this notebook, we’ll have a solid foundation for building tools, and we’ll be ready to integrate them into an agent in the next notebook.


### ✅ Example 1: list_files
A simple tool that requires no input. It just lists files in a folder.

In [ ]:
{
  "name": "list_files",
  "description": "Lists all files in the current working directory.",
  "parameters": {
    "type": "object",
    "properties": {},
    "required": []
  }
}


### 🔍 Python Code Version

In [ ]:
import os
import json

# 🔹 Step 1: Imports and Setup
source_dir = "/content/docs_folder"

# Make sure the directory exists
if not os.path.exists(source_dir):
    raise FileNotFoundError(f"📁 Directory not found: {source_dir}")

# List and build full file paths
file_list = [
    os.path.join(source_dir, f)
    for f in os.listdir(source_dir)
    if os.path.isfile(os.path.join(source_dir, f))
]

## 🧠 These Aren’t One-to-One

Python code **does** the task.
Tool JSON **describes** the task in a way the LLM can recognize and invoke it.

Think of the tool JSON as **an API spec**, not an implementation.

You're uncovering some of the most important (and often misunderstood) differences between *traditional code* and *tool design for agents*. Let’s walk through your points and connect the dots:

---

## 🧩 1. Why `"type"`, `"properties"`, `"required"`?

These fields come from **JSON Schema**, which is a standard format used to describe data structures.
OpenAI uses JSON Schema so the LLM can:

* understand what arguments a tool accepts,
* validate them,
* and generate calls in the correct structure.

Let’s explain each:

| JSON Schema Field  | What It Means                                                | Python Equivalent                                                 |
| ------------------ | ------------------------------------------------------------ | ----------------------------------------------------------------- |
| `"type": "object"` | This tool expects an object (i.e., a dictionary) as input    | A function that takes named arguments                             |
| `"properties"`     | Lists the possible fields/parameters inside the input object | Function parameters like `file_name: str`                         |
| `"required"`       | Specifies which parameters must be included                  | Equivalent to **non-default function args** (i.e. not `**kwargs`) |

### Why not use `"args"`?

Because `"args"` and `"kwargs"` are Python-specific.
JSON Schema is **language-agnostic**, and OpenAI tools aim to be platform-neutral.

---

## 📁 2. Where’s `source_dir` in the tool?

Great observation. It’s **abstracted away**.

The tool description:

```json
{
  "name": "list_files",
  "description": "Lists all files in the current working directory.",
  "parameters": {
    "type": "object",
    "properties": {},
    "required": []
  }
}
```

...assumes that:

* The **agent runtime environment** has already defined a `source_dir`.
* Or, the tool itself (on the backend) is **hardcoded** to know where to look.
* Or the agent is *managing context* so `cwd = /content/docs_folder`.

So yes, you’re right: **the LLM is trusting that the tool does what it says**.
It doesn’t see `os.listdir()` or `source_dir` — it just learns:

> “When I want to get a list of files, call `list_files` with no parameters.”

The *developer* is responsible for wiring up the actual behavior.

---

## 🧠 3. Isn't This Asking Too Much of the LLM?

That’s an insightful concern, and here’s the truth:

### ✅ Yes, we're asking a lot — but intentionally.

Think of it like this:

| Traditional Code                  | Agent-Oriented Approach                            |
| --------------------------------- | -------------------------------------------------- |
| You define everything             | You *describe* capabilities                        |
| You write imperative instructions | You delegate tool selection to the LLM             |
| Focus is on implementation        | Focus is on *orchestration* and intent recognition |
| LLM is a text generator           | LLM is a **planner** and **dispatcher**            |

The LLM is **not building the tool on the fly** — it’s choosing **which tool** to call and **what arguments** to provide. That’s a very different job than execution.

The actual execution still happens in your backend Python code — the LLM just produces a JSON like:

```json
{
  "tool_name": "list_files",
  "args": {}
}
```

Then your app does:

```python
if tool_name == "list_files":
    return list_files()
```

---

## 🧠 Why All This?

Because this pattern **scales**.

Once you define tools:

* The LLM can choose, combine, and sequence them
* You can add, update, or swap tools without retraining anything
* You separate **intelligence** (LLM) from **execution** (tools)

---

## ✅ TL;DR – What’s Going On?

You're designing a **modular architecture** where:

* The **LLM plans** ("What should I do?")
* The **tools act** ("Do this")
* Your **code glues them together**

So yes — it’s different from writing raw Python.
But it enables automation that’s general, adaptive, and reusable.




## ✅ What’s Happening at Each Layer

### 1. **JSON Tool Schema** (For the LLM)

This is:

* **Not Python**
* A **structured description** of what a tool *does*
* What the **LLM reads** to understand:

  * "What tools are available?"
  * "What arguments can I provide?"
  * "What is this tool used for?"

💡 Think of this as an *API contract* between the LLM and your Python code.

---

### 2. **Python Tool Function** (For Execution)

This is:

* Real Python code (your actual tool logic)
* Executes the task — e.g., list files, read a file, summarize content
* Matches the interface defined in the JSON

🧩 Your job is to **make sure the Python function accepts the inputs** the LLM defines via the JSON schema.

---

### 3. **LLM Role**

The LLM:

* Reads the available tool schemas (via prompt)
* Decides: “Ah, based on the user request, I should call `read_file` with `filename: 'lecture_01.txt'`”
* Outputs a **JSON invocation** like:

```json
{
  "tool_name": "read_file",
  "args": { "filename": "lecture_01.txt" }
}
```

---

### 4. **Your Agent Runtime**

Your agent:

* Parses the LLM’s output
* Extracts `tool_name` and `args`
* Calls the corresponding Python function like:

```python
read_file(filename="lecture_01.txt")
```

---

## 🧠 In Summary:

| Layer       | Role          | Format                    | Purpose                     |
| ----------- | ------------- | ------------------------- | --------------------------- |
| Tool Schema | For the LLM   | JSON (JSON Schema format) | Describe tool structure     |
| Tool Code   | For your app  | Python                    | Actually does the work      |
| Tool Call   | From the LLM  | JSON (tool\_name + args)  | Tells your code what to run |
| Execution   | By your agent | Python                    | Calls the real function     |



## Who Builds the Tool?

**You build the actual tool** (in Python), and the **LLM simply chooses when and how to use it.**

---

## 🔧 Your Role:

You are the **tool builder**. This means:

* Writing the actual Python functions that do the work (`read_file()`, `summarize_text()`, etc.)
* Defining JSON schema metadata to describe the tool to the LLM
* Making sure the inputs the LLM is allowed to use (via JSON) align with the parameters your function accepts

---

## 🤖 The LLM’s Role:

The LLM is the **tool orchestrator**. It:

* Reads your schema definitions
* Interprets the user request
* Decides: “Ah, I should call `read_file` with `filename='foo.txt'`”
* Outputs a structured call
* Doesn’t know or care *how* the tool works — just what it’s allowed to do

---

### 🔁 Example Flow

You write this:

```python
def read_file(filename: str):
    with open(filename, "r") as f:
        return f.read()
```

You define this JSON:

```json
{
  "name": "read_file",
  "description": "Reads the content of a file from disk.",
  "parameters": {
    "type": "object",
    "properties": {
      "filename": {
        "type": "string",
        "description": "The name of the file to read"
      }
    },
    "required": ["filename"]
  }
}
```

The LLM sees this and says:

> "To answer the user, I should use `read_file` with `filename='doc_001.txt'`."

Then your agent calls:

```python
read_file(filename="doc_001.txt")
```




Let's walk through building a complete tool, step-by-step — **both the Python code and the JSON schema**, side-by-side.

---

## ✅ Build a `count_words` tool

It will take a text file and return the number of words in it.

---

### 🧠 Step 1: Define the Python Tool (the actual function)

```python
def count_words(filename: str) -> int:
    with open(filename, 'r') as f:
        content = f.read()
    return len(content.split())
```

✅ **What this does**:

* Takes in a file path
* Reads the file content
* Splits it into words
* Returns the word count

---

### 🔧 Step 2: Define the Tool Schema (what the LLM sees)

```json
{
  "name": "count_words",
  "description": "Counts the number of words in a specified text file.",
  "parameters": {
    "type": "object",
    "properties": {
      "filename": {
        "type": "string",
        "description": "The name of the file to count words in"
      }
    },
    "required": ["filename"]
  }
}
```

✅ **What this tells the LLM**:

* The tool is called `count_words`
* It expects a single parameter called `filename` (a string)
* It doesn’t know how it works, just what it does and what it needs

---

### 🔁 Step 3: What the LLM Might Output (based on a user request)

If the user says:

> “How many words are in `lecture_notes.txt`?”

The LLM might output:

```json
{
  "tool_name": "count_words",
  "args": {
    "filename": "lecture_notes.txt"
  }
}
```

Your orchestrator would then call:

```python
count_words("lecture_notes.txt")
```

---

### 🎯 Recap

| Component          | Your Responsibility        | LLM’s Responsibility           |
| ------------------ | -------------------------- | ------------------------------ |
| Python function    | Write real logic in Python | —                              |
| JSON schema        | Describe inputs + purpose  | Understand how to use the tool |
| Agent orchestrator | Handle tool execution      | —                              |
| LLM                | Decide when to use tool    | Choose tool and fill arguments |




## 🤖 LLM-Orchestrated Agents = **Chat UX + Code API**

### 🔷 What *you* do (as the developer):

1. **Write the tools** → Python functions that do specific things well
2. **Describe the tools** → JSON specs that help the LLM understand what they are and how to use them
3. **Build the agent loop** → A system that routes LLM requests into function calls

### 🔷 What the *LLM* does (as the orchestrator):

1. **Understands the user request**
2. **Chooses the right tool(s)** based on natural language and schema
3. **Fills in the tool arguments** using reasoning
4. **Delegates execution to code** (it doesn’t run the tool — it just picks and preps)
5. **Processes the output** and continues the conversation

---

## 💡 Why it works so well

* **LLMs are good at reasoning and language**, but not running code
* **Python is good at execution**, but not interpreting vague instructions
* This pattern **bridges both worlds**, letting each do what it does best

---

## 🧠 Your takeaway

> You're not making the LLM do everything.
>
> You're building a system where **LLM = brain**, **Python = hands**.

This separation of concerns is what makes agents:

* Cheaper 💸 (LLM doesn’t have to learn to do everything)
* Faster ⚡ (code runs tools directly)
* Safer ✅ (tools are verified, tested, limited in scope)




Let’s build a slightly more advanced tool next — one that:

✅ Has multiple parameters
✅ Expects inputs from the LLM
✅ Could be conditionally used depending on the user request

---

## 🛠️ Tool Concept: `search_documents`

This tool searches for a keyword or phrase in a set of documents and returns matching filenames.

---

### 🧪 Step 1: Write the **actual Python function**

```python
import os

def search_documents(query: str, folder: str = "/content/docs_folder"):
    results = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        if os.path.isfile(filepath):
            with open(filepath, "r", encoding="utf-8") as f:
                content = f.read()
                if query.lower() in content.lower():
                    results.append(filename)
    return results
```

* ✅ This code works.
* ✅ It's testable independently of the LLM.
* ✅ It only does one thing well — that’s what you want from tools.

---

### 🧾 Step 2: Define the **tool schema** in JSON format

This tells the LLM **what** the tool does, and what arguments it needs:

```json
{
  "name": "search_documents",
  "description": "Searches all documents in the folder for a given keyword or phrase.",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {
        "type": "string",
        "description": "The keyword or phrase to search for in the documents."
      },
      "folder": {
        "type": "string",
        "description": "The folder path to search in (default is '/content/docs_folder')."
      }
    },
    "required": ["query"]
  }
}
```

### 🔍 Notice:

* `query` is **required**
* `folder` is **optional** (we’ll default to a value in the Python code)
* The LLM now understands how to **call this tool** like:

```json
{
  "tool_name": "search_documents",
  "args": {
    "query": "vector database"
  }
}
```

---

## 🧠 Why this matters

You now have a tool that the LLM can choose to use when a user asks things like:

* “Which files talk about vector databases?”
* “Search all my notes for mentions of LangChain.”
* “Do any documents explain agent loops?”

You’re not manually mapping user intent → tool calls anymore.
**The LLM handles that orchestration.**






# 🧰 OpenAI “Tools” (Function Calling): What the API expects

**OpenAI’s API defines the shape** when you use *tool/function calling*. You send a tool schema, and the model replies with an **assistant message** that contains a `tool_calls` array. Each item has a function `name` and an `arguments` JSON string. Example of what you actually read off the SDK:

```json
{
  "role": "assistant",
  "tool_calls": [
    {
      "type": "function",
      "function": {
        "name": "search_documents",
        "arguments": "{\"query\":\"vector database\"}"
      }
    }
  ]
}
```

That structure (including `tool_calls → function → name/arguments`) is OpenAI-specific and documented in their tool/function calling guide.

* Your example

  ```json
  {"tool_name":"search_documents","args":{"query":"vector database"}}
  ```

  is a **teaching abstraction** (or a prompt-only fallback). It’s fine if you’re not using tool calling, but when you *do* use tool calling, you don’t need code fences or custom wrappers—the SDK gives you the call in `message.tool_calls`, and you `json.loads` the `arguments`.

* **Other platforms differ.** Anthropic, Gemini, etc., have their own shapes (same concept: name + JSON args; different field names). Azure OpenAI mirrors OpenAI’s shape closely, since it’s the same API surface hosted in Azure.

* If you want to **force strict JSON** for non-tool outputs (like summaries), OpenAI also has “Structured Outputs”/JSON Schema support to guarantee valid JSON—again, an API-defined contract.

**Bottom line:**
Yes—the exact return format depends on the provider. With OpenAI tool calling, the “tool call object” shape is determined by OpenAI; your schema influences the **`arguments`** fields, not the outer envelope. You can still normalize it into your own `{tool_name, args}` shape internally if you prefer, but you don’t have to.

* 1: https://platform.openai.com/docs/guides/function-calling?utm_source=chatgpt.com "Function calling - OpenAI API"
* 2: https://platform.openai.com/docs/guides/structured-outputs?utm_source=chatgpt.com "Structured model outputs - OpenAI API"


## 1) What you SEND (request)

* Add a `tools` array. Each tool is type `function` with a **name**, **description**, and **parameters** (a JSON Schema).
  Example:

  ```python
  tools = [{
    "type": "function",
    "function": {
      "name": "search_documents",
      "description": "Searches all documents in the folder for a given keyword or phrase.",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {"type": "string", "description": "Keyword or phrase."},
          "folder": {"type": "string", "description": "Folder path"}
        },
        "required": ["query"]
      }
    }
  }]
  ```

  You pass this via `tools=tools` in your API call. The model uses the schema to decide if and how to call the function. ([OpenAI Platform][1])

* (Optional) **Control tool use** with `tool_choice`:

  * `"auto"` → model may call a tool,
  * `"none"` → model won’t,
  * or a specific function to force exactly that tool.
    There’s also **`allowed_tools`** to restrict to a subset. ([OpenAI Platform][1])

* (Optional) **Structured Outputs (strict)**: you can make the model **strictly** follow your function schema by enabling structured outputs for tools; this increases reliability of the `arguments`. ([OpenAI][2], [OpenAI Platform][3])

## 2) What you GET BACK (response)

* The assistant message may include **`tool_calls`** (an array). Each item has:

  * `function.name` → the tool to run
  * `function.arguments` → **a JSON string** you must `json.loads`
* You then **execute** your Python function with those args. ([OpenAI Platform][1])

## 3) How to RETURN tool results to the model

* With **Chat Completions**, you send another API call with an appended **`role: "tool"`** message that includes:

  * `tool_call_id` (echo the id from the tool call),
  * the tool **output** as message `content`.
* The model can then read those results and continue (e.g., summarize, decide next step). ([OpenAI Platform][1])

---

# ✅ Practical rules & gotchas

* **Schema is a contract.** Use JSON Schema features (`type`, `required`, `enum`, `minItems`, etc.). This helps the model propose correct args and helps *you* validate before running code. ([OpenAI Platform][1])
* **Arguments are JSON.** The `arguments` you parse are a JSON string. Always `json.loads` and then **validate** (Pydantic/JSON Schema) before executing. ([OpenAI Platform][1])
* **Keep tools focused.** One clear purpose per tool and concise descriptions improve accuracy (and reduce tokens). ([OpenAI Platform][4])
* **Prefer constraints over prose.** If your API/model supports **Structured Outputs** (strict), enable it to enforce your schema for function arguments. It materially improves reliability over prompt-only formatting. ([OpenAI][2], [OpenAI Platform][3])
* **Latency & cost.** Tool definitions live in the prompt context; keep names/descriptions compact. Use `tool_choice`/`allowed_tools` to reduce unnecessary calls. ([OpenAI Platform][5])
* **Multiple tool calls.** Models can propose **more than one** call in a turn (parallel or sequential). Your code should loop through `tool_calls` and respond with one `role:"tool"` message per call (each with its own `tool_call_id`). ([OpenAI Platform][1])
* **Large inputs.** Don’t stuff big blobs into `arguments`. Pass references (file IDs/paths) and let your Python do the I/O, or use built-ins like **File Search** / **Actions** when appropriate. ([OpenAI Platform][6])

---

# 🔧 Minimal Python pattern (Chat Completions)

```python
from openai import OpenAI
import json

client = OpenAI()

messages = [{"role": "user", "content": "Find mentions of 'vector database' in my docs"}]

resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=[{
        "type": "function",
        "function": {
            "name": "search_documents",
            "description": "Search docs for a keyword/phrase.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"},
                    "folder": {"type": "string"}
                },
                "required": ["query"]
            }
        }
    }],
    tool_choice="auto"
)

msg = resp.choices[0].message
for call in msg.tool_calls or []:
    name = call.function.name
    args = json.loads(call.function.arguments)

    # run your Python tool safely (validate args first!)
    result = search_documents(**args)

    # send tool result back
    messages.append({
        "role": "assistant",
        "content": None,
        "tool_calls": [call]  # keep the call in history if you want
    })
    messages.append({
        "role": "tool",
        "tool_call_id": call.id,
        "name": name,
        "content": json.dumps({"results": result})
    })

# then call the model again to let it use the tool result(s)
final = client.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(final.choices[0].message.content)
```

*(Shape and field names follow OpenAI’s tool/function calling guides and API reference.)* ([OpenAI Platform][1])

---

## TL;DR

* **Define** tools via `tools=[{"type":"function","function":{name, description, parameters}}]`.
* **Receive** tool calls in `message.tool_calls[*].function.arguments` (JSON string).
* **Execute** your code, then **reply** with a `role:"tool"` message (including the `tool_call_id`).
* Use **Structured Outputs** (strict) + **validation** to maximize reliability; keep schemas deliberate and compact. ([OpenAI Platform][1], [OpenAI][2])

If you want, I can tailor a version of this snippet to your exact `search_documents` function and folder layout.

[1]: https://platform.openai.com/docs/guides/function-calling?utm_source=chatgpt.com "Function calling - OpenAI API"
[2]: https://openai.com/index/introducing-structured-outputs-in-the-api/?utm_source=chatgpt.com "Introducing Structured Outputs in the API"
[3]: https://platform.openai.com/docs/guides/structured-outputs?utm_source=chatgpt.com "Structured model outputs - OpenAI API"
[4]: https://platform.openai.com/docs/guides/tools?utm_source=chatgpt.com "Using tools - OpenAI API"
[5]: https://platform.openai.com/docs/api-reference/introduction?utm_source=chatgpt.com "API Reference"
[6]: https://platform.openai.com/docs/guides/tools-file-search?utm_source=chatgpt.com "File search - OpenAI API"


Let’s now build a tool that accepts **multiple argument types**, including a **boolean** and a **list**. This adds more flexibility and reflects real-world use cases agents often need.

---

## 🛠️ Tool Concept: `filter_documents`

This tool filters documents based on their filename and/or content, optionally returning only those that include **all** the keywords provided.

---

### 🧪 Step 1: Python Function

```python
import os

def filter_documents(keywords: list, require_all: bool = False, folder: str = "/content/docs_folder"):
    matched_files = []

    for filename in os.listdir(folder):
        if not filename.endswith(".txt"):
            continue
        path = os.path.join(folder, filename)
        if not os.path.isfile(path):
            continue

        with open(path, "r", encoding="utf-8") as f:
            content = f.read().lower()

        checks = [kw.lower() in content for kw in keywords]

        if (require_all and all(checks)) or (not require_all and any(checks)):
            matched_files.append(filename)

    return matched_files
```

* ✅ `keywords`: list of strings to match
* ✅ `require_all`: a boolean for AND vs OR matching
* ✅ Flexible and simple

---

### 🧾 Step 2: Tool Schema for the LLM

```json
{
  "name": "filter_documents",
  "description": "Filters documents based on one or more keywords in their content. Can match all or any.",
  "parameters": {
    "type": "object",
    "properties": {
      "keywords": {
        "type": "array",
        "items": {
          "type": "string"
        },
        "description": "List of keywords to match in the document content."
      },
      "require_all": {
        "type": "boolean",
        "description": "If true, all keywords must match; if false, any match is accepted."
      },
      "folder": {
        "type": "string",
        "description": "The folder path to search in (default is '/content/docs_folder')."
      }
    },
    "required": ["keywords"]
  }
}
```

### 🔍 Now the LLM can do things like:

```json
{
  "tool_name": "filter_documents",
  "args": {
    "keywords": ["agent", "action", "memory"],
    "require_all": true
  }
}
```

---

## 🧠 Why this is powerful

* The LLM gets **structured control**: it knows how to call this tool correctly.
* You get to **enforce safety and input types** through schema.
* Booleans and lists open up a whole new level of query logic.



**You should design tools with provider-specific API requirements in mind**. Different LLM providers expose *very similar concepts* (name + JSON args + schema), but the **envelope and field names differ**. The safest approach is:

* **Provider-agnostic core**: your Python function(s) with clear signatures and validation.
* **Thin adapters** per provider (OpenAI, Anthropic, Gemini) that translate between that provider’s tool-calling shape and your core.


---

# 🧰 Tool-Calling API requirements (what to watch for)

## 1) What’s common across providers

* You **declare tools** with a **name**, short **description**, and **JSON Schema** for inputs.
* The model may return a **tool call**: *(tool name + JSON arguments)*.
* You **execute** your Python code and **send the result back** to the model so it can continue.

## 2) Provider differences (envelopes & fields)

| Provider               | How you declare tools (request)                                                       | What the model returns                                    | Where args live                                                                          |
| ---------------------- | ------------------------------------------------------------------------------------- | --------------------------------------------------------- | ---------------------------------------------------------------------------------------- |
| **OpenAI**             | `tools=[{"type":"function","function":{name, description, parameters(JSON Schema)}}]` | Assistant message with `tool_calls[]` (possibly multiple) | `tool_calls[i].function.arguments` (JSON string you `json.loads`) ([OpenAI Platform][1]) |
| **Anthropic (Claude)** | `tools=[{name, description, input_schema(JSON Schema)}]`                              | A `tool_use` block with `name` and `input`                | `content[].tool_use.input` (already structured JSON) ([Anthropic][2])                    |
| **Google Gemini**      | Function declarations in the request/session                                          | A **function call** with parameters                       | Arguments appear as structured params in the call object ([Google AI for Developers][3]) |

> Bottom line: same idea, **different field names/locations**. Build an adapter that normalizes each provider’s call into `{tool_name, args}` before dispatching.

## 3) Reliability features you can (and should) use

* **Structured/Strict outputs** (JSON Schema enforcement) for non-tool responses and/or tool args where supported. This boosts correctness of JSON and types. Still validate in your code. ([OpenAI Platform][4], [OpenAI][5])
* **Tool selection controls**: e.g., OpenAI’s `tool_choice` to force or forbid a tool call in a turn. ([OpenAI Platform][1])
* **Multiple tool calls in one turn**: handle arrays; loop over calls and return one tool result per call (OpenAI uses `tool_call_id`). ([OpenAI Platform][1])

## 4) Practical requirements & guardrails (provider-agnostic)

* **Schema as contract**: keep it tight; mark required fields; use enums for tool names/choices. Validate with Pydantic/JSON Schema before execution. ([OpenAI Platform][4])
* **Argument size**: pass **references** (file paths/IDs), not big blobs, to avoid token bloat. ([OpenAI Platform][1])
* **Error handling**: fail fast on validation; **retry ≤ 1–2** with stricter instructions; otherwise return a safe error.
* **Security**: treat model output as untrusted input; whitelist tools; check paths, URLs, and ranges; make side-effecting tools **idempotent**.
* **Observability**: log raw tool calls, validation errors, retry counts; include a `schema_version` to evolve safely.

## 5) Architecture tip (ports & adapters)

* **Core (provider-agnostic)**:

  * `search_documents(query: str, folder: str) -> list[str]`
  * Pydantic models for args/results.
* **Adapters**:

  * **OpenAI adapter**: read `tool_calls[*].function.name/arguments`, `json.loads`, validate → call core → send `role:"tool"` message with `tool_call_id`. ([OpenAI Platform][1])
  * **Anthropic adapter**: read `tool_use` block `input` → validate → call core → reply with `tool_result`. ([Anthropic][6])
  * **Gemini adapter**: read function call params → validate → call core → send `FunctionResponse`. ([Google AI for Developers][7])

---

**Answering your direct question:**
You should **always** keep the **API requirements of the platform you’re using** in mind (OpenAI in your course), because they determine the request fields, the response envelope, and how you return tool results. If you plan to be multi-provider, isolate those differences in small adapters and keep your tool code and validation provider-agnostic.

[1]: https://platform.openai.com/docs/guides/function-calling?utm_source=chatgpt.com "Function calling - OpenAI API"
[2]: https://docs.anthropic.com/en/docs/agents-and-tools/tool-use/overview?utm_source=chatgpt.com "Tool use with Claude"
[3]: https://ai.google.dev/gemini-api/docs/function-calling?utm_source=chatgpt.com "Function calling with the Gemini API | Google AI for Developers"
[4]: https://platform.openai.com/docs/guides/structured-outputs?utm_source=chatgpt.com "Structured model outputs - OpenAI API"
[5]: https://openai.com/index/introducing-structured-outputs-in-the-api/?utm_source=chatgpt.com "Introducing Structured Outputs in the API"
[6]: https://docs.anthropic.com/en/docs/agents-and-tools/tool-use/implement-tool-use?utm_source=chatgpt.com "How to implement tool use"
[7]: https://ai.google.dev/gemini-api/docs/live-tools?utm_source=chatgpt.com "Tool use with Live API | Gemini API | Google AI for Developers"
